This code analyzes the mean flourescence intensity of the red channel at each time point, to calculate the rate of increase in the red fluorescence of the dye, which signals the rate of mitochondrial ROS production.

Import Libraries

In [1]:
import czifile
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk
from scipy.ndimage import binary_fill_holes
from matplotlib.colors import LogNorm
from skimage import exposure, filters, measure
import scipy.ndimage as ndi
import pandas as pd
import os
from scipy import stats

Define Sub Functions

In [2]:
def analyze_time_course_image(image_path):
    czi_file = czifile.CziFile(image_path)
    czi_data = czi_file.asarray()
    all_timepoints = slice(None)
    green_channel_data = czi_data[all_timepoints, :, :, :, :]
    green_channel_data = np.squeeze(green_channel_data)
    intensity_list = []
    times = []
    for frame_index in range(green_channel_data.shape[0]):
        green_channel = green_channel_data[frame_index, :, :]
        times.append(frame_index)
        mean_intensity_channel1 = np.mean(green_channel)
        intensity_list.append(mean_intensity_channel1)
    return intensity_list, times

Define Main Function

In [3]:
all_data = []

folder_path = "test_images"
for well_image in os.listdir(folder_path):
    if well_image.lower().endswith(".czi"):  # Filter for CZI files
        well_image_path = os.path.join(folder_path, well_image)
        well_image_base_name = os.path.basename(well_image)[:-4]
        intensity_list, times = analyze_time_course_image(well_image_path)
        df = pd.DataFrame({
            "Filename": [well_image_base_name] * len(intensity_list),
            "Time": times,
            "MFI": intensity_list
            })
        
        all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined DataFrame to a single CSV
combined_csv_path = os.path.join("MITOTRACKER_ROS.xlsx")
combined_df.to_excel(combined_csv_path, index=False)